In [42]:
import pandas as pd

In [44]:
df=pd.read_csv("spotify_millsongdata.csv")

In [45]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [46]:
df.shape

(57650, 4)

In [50]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [52]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [53]:
df.tail(5)

,artist,song,text
4995,Oasis,Going Nowhere,Hate the way that you've taken back \r\nEvery...
4996,Avril Lavigne,What The Hell,You say that I'm messing with your head \r\nY...
4997,Keith Urban,Freedom's Finally Mine,Freedom's Finally Mine \r\nI'm on a red dirt ...
4998,Garth Brooks,Hard Luck Woman,If never I'd met you \r\nI'd never have seen ...
4999,Steely Dan,A Little With Sugar,I remember Christmas morning \r\nBack in 1954...


In [54]:
df['text'][0]

"I stopped flying a long time ago,  \r\nThinking that's just for the birds and bees.  \r\nBut when I look up (I look up)  \r\nAnd see the sky, (see the sky)  \r\nI like the blue and I know why.  \r\n  \r\nYou came from Zeus, we come from Uranus,  \r\nHoping to meet soul to soul.  \r\nWe see you clearly, (clearly)  \r\nWith our third eye, (with our third eye)  \r\nThere's no more fear and I know why.  \r\n  \r\nSky people, that's what we are,  \r\nSky people, that's what we are.  \r\nOne day we'll fly and leap through the sky  \r\nTo look for a good land hand in hand.  \r\n  \r\nSky people, that's what we are,  \r\nSky people, that's what we are,  \r\nSky people, that's what we are,  \r\nSky people, that's what we are.\r\n\r\n"

In [55]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [56]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [63]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [64]:
similarity[0]

array([1.        , 0.0078338 , 0.03838586, ..., 0.04031039, 0.0074373 ,
       0.0246232 ])

In [66]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [67]:
def recommendation(song_df):
    try:
        idx = df[df['song'] == song_df].index[0]
    except IndexError:
        print(f"Error: Song '{song_df}' not found in the dataset.")
        return []

    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

    songs = []
    for i in distances[1:6]:
        songs.append(df.iloc[i[0]]['song'])
    return songs


In [90]:
recommendation('Goodbye Stranger')

['Goodbye To Romance',
 'Goodbye Love',
 'Long, Long Way To Go',
 "I Just Can't Let You Say Goodbye",
 'It Takes So Long (To Say Goodbye)']

In [96]:
df.sample(5)

,artist,song,text
264,Morrissey,I've Changed My Plea To Guilty,i'm standing in the dock \r with my innocent ...
2471,Barbra Streisand,Ding-dong! The Witch Is Dead!,[barbra:] \r once there was a wicked witch \...
1764,Michael W. Smith,Great Is The Lord,"great is the lord, \r he is holy and just \r..."
158,Phil Collins,Why Can't It Wait 'till Morning,why can't it wait 'til morning? \r we can tal...
720,Ween,Awesome Sound,got an awesome sound goin' down \r got an awe...


In [109]:
import pickle

# Save similarity matrix
pickle.dump(similarity, open("similarity.pkl", "wb"))

# Save dataframe
pickle.dump(df, open("df.pkl", "wb"))
